In [1]:
from aie_kernel_builder import AIEKernelBuilder_2in_1out
import numpy as np

In [2]:
%%kernel
#include "ipulog.h"

void vecadd(uint8_t* a, uint8_t* b, uint8_t* c, uint8_t* stdout) {

    IPULogger log(stdout, 256);
    ::aie::vector<uint8_t, 32> ai, bi, ci;

    log.write("Starting Kernel Execution!\n");

    aie::tile tile=aie::tile::current();
    uint64_t Tstart = tile.cycles(); //cycle counter of the AI Engine tile

    for(int j=0; j<4096; j+=32) {
        ai = ::aie::load_v<32>(a);
        a += 32;
        bi = ::aie::load_v<32>(b);
        b += 32;
        ci = ::aie::add(ai, bi);
        ::aie::store_v(c, ci);
        c += 32;
    }

    uint64_t Tend = tile.cycles(); //cycle counter of the AI Engine tile
    uint64_t cycles = Tend - Tstart;

    log.write("Completed executing. cycles=%u\n", cycles);
}

In [3]:
a = np.random.randint(0, 256, size=4096, dtype=np.uint8)
b = np.random.randint(0, 256, size=4096, dtype=np.uint8)
c = np.zeros(4096, dtype=np.uint8)

vecadd.c.array = c
vadd_test = AIEKernelBuilder_2in_1out(vecadd, a, b, c)

In [ ]:
ans = vadd_test.run(cache=False)

Using cached vecadd kernel object file...
Building the xclbin...


In [ ]:
ans